# Rule Fit

Implement a Rule Fit model using the rulefit library

Features:
- 6 meteorological features: Pressure, Relative Vorticity 850hPa, Wind Gust, Sea Surface Temperature (1000hPa), Wind Speed 850hPa, Air Density
- Samples only from Z11
- Samples only from today - previous days samples not considered

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
if os.getcwd() != '/content/drive/My Drive/Tropical_Cyclones_Thesis/ERA5_Dataset':
  os.chdir('./drive/MyDrive/Tropical_Cyclones_Thesis/ERA5_Dataset')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime
from datetime import timedelta

np.random.seed(42)

from sklearn.model_selection import train_test_split
from sklearn import metrics

# Loading Data

In [3]:
df_era5 = pd.read_csv('./DATASET_COMPLETED/ERA5_16zones_avg_std_10D_NoWindDirection.csv')
df_era5 = df_era5.set_index('DATE')
df_era5 = df_era5.drop(columns=['S.IndAll', 'S.IndGen'])
df_era5 = df_era5[df_era5.index > '1980-02']
df_era5 = df_era5[df_era5.index < '2022']

In [4]:
from datetime import datetime


def is_exact_substring(main_string, substring):
    # Check if substring is in the main_string
    if substring in main_string:
        # Check if the substring is not followed by a digit
        index = main_string.find(substring)
        if index + len(substring) == len(main_string) or not main_string[index + len(substring)].isdigit():
            return True
    return False

def filter_features(df, feats):
  columns = df.columns
  selected_columns = []
  for col in columns:
    for feat in feats:
      if is_exact_substring(col, feat):
        selected_columns.append(col)
  return df[selected_columns]

def day_of_year(date_string):
    date_object = datetime.strptime(date_string, '%Y-%m-%d')
    return date_object.timetuple().tm_yday

selected_meteo_feat = ['P_', 'Vor_850hPa_', 'Wind_Gust_', 'Wind_1000hPa', 'Wind_850hPa', 'Wind_300hPa', 'T_1000hPa', 'Air_Density']
#selected_meteo_feat = ['P_', 'Vor_850hPa_', 'Wind_Gust_', 'Wind_850hPa', 'T_1000hPa', 'Air_Density']

selected_steps = ['-0']#, '-1', '-2', '-3']
selected_zones = ['_Z11']

print(8*4*6*2)

df_filtered = filter_features(df_era5, selected_meteo_feat)
df_filtered = filter_features(df_filtered, selected_steps)
df_filtered = filter_features(df_filtered, selected_zones)

cols = []
for elem in df_filtered.columns:
  cols.append(elem.replace('_Z11-0',''))
df_filtered.columns = cols

# Convert to list of day numbers
day_numbers = [day_of_year(date_str) for date_str in df_filtered.index]
df_filtered['yday'] = day_numbers

df_filtered

384


,P_Mean,P_Std,Vor_850hPa_Mean,Vor_850hPa_Std,Wind_Gust_Mean,Wind_Gust_Std,Wind_1000hPa_Mean,Wind_1000hPa_Std,Wind_850hPa_Mean,Wind_850hPa_Std,Wind_300hPa_Mean,Wind_300hPa_Std,T_1000hPa_Mean,T_1000hPa_Std,Air_Density_Mean,Air_Density_Std,yday
DATE,,,,,,,,,,,,,,,,,
1980-02-01,100839.731017,181.191942,-1.326374e-05,0.000030,9.161096,1.891302,8.030626,2.043022,8.482522,2.708363,5.320485,2.539159,298.471739,0.587952,1.160672,0.002807,32
1980-02-02,100819.805114,275.843829,-1.415175e-05,0.000042,13.511941,2.658089,10.984181,2.337932,13.300693,3.685055,9.385830,2.922391,298.590989,0.560706,1.159397,0.003655,33
1980-02-03,100624.544895,484.595148,-3.241318e-05,0.000063,14.583778,3.584316,12.698438,3.631855,15.776457,6.220648,9.869005,5.341686,298.370429,0.675645,1.158297,0.006810,34
1980-02-04,100803.244697,515.555038,-8.678676e-06,0.000079,13.257321,5.089852,10.006286,4.507314,11.668562,8.944703,9.528795,3.824716,298.688186,0.982700,1.158986,0.008253,35
1980-02-05,100982.396277,245.186655,5.546546e-06,0.000018,10.703720,3.427292,8.347398,2.595995,7.948358,5.141756,14.101187,3.314044,298.337832,0.897690,1.162109,0.005853,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,101312.753918,37.618223,5.143506e-06,0.000021,4.121407,1.500981,2.490474,1.160273,2.725058,1.919368,9.128163,5.538250,298.534718,0.575989,1.164909,0.001809,361
2021-12-28,101220.225639,49.067544,-2.528924e-07,0.000024,4.851551,3.032916,3.386623,3.012908,2.221736,1.332991,13.674474,7.572192,298.337200,0.893729,1.165014,0.003450,362
2021-12-29,101050.137778,77.453997,-1.208212e-05,0.000062,8.444925,4.404138,5.737394,3.843089,4.733215,1.904213,14.903417,6.812252,297.744687,1.149675,1.165850,0.005070,363


In [5]:
df_target = pd.read_csv('./DATASET_COMPLETED/old_dataset/ibtracs_Z11.csv')
df_target = df_target.drop(columns=['Unnamed: 0'])
df_target = df_target.set_index('DATE')
df_target = df_target[df_target.index > '1980-02']
df_target = df_target[df_target.index < '2022']
df_target = df_target['TC_PRESENCE']

df_target

DATE
1980-02-01    0
1980-02-02    1
1980-02-03    1
1980-02-04    0
1980-02-05    0
             ..
2021-12-27    0
2021-12-28    0
2021-12-29    0
2021-12-30    0
2021-12-31    0
Name: TC_PRESENCE, Length: 15310, dtype: int64

# Train-Test Splitting

In [6]:
#################################################################
###################### TRAINING SET #############################
df_X_train = df_filtered[df_filtered.index < '2012']
df_y_train = df_target[df_target.index < '2012']
X_train = df_X_train.values
y_train = df_y_train.values
y_train = np.where(y_train > 0, 1, y_train) # If S.IndAll is greater than 0 -> a cyclone is present
y_train = y_train.reshape(1,-1)[0]

#################################################################
######################### TEST SET ##############################
df_X_test = df_filtered[df_filtered.index > '2012']
df_y_test = df_target[df_target.index > '2012']
X_test = df_X_test.values
y_test = df_y_test.values
y_test = np.where(y_test > 0, 1, y_test) # If S.IndAll is greater than 0 -> a cyclone is present
y_test = y_test.reshape(1,-1)[0]

print('TRAINING: ', X_train.shape, y_train.shape)
print('    ---> Positive Class: ', np.count_nonzero(y_train))
print('TESTING: ', X_test.shape, y_test.shape)
print('    ---> Positive Class: ', np.count_nonzero(y_test))

TRAINING:  (11657, 17) (11657,)
    ---> Positive Class:  532
TESTING:  (3653, 17) (3653,)
    ---> Positive Class:  169


# Rule Fit Classifier

In [7]:
! pip install rulefit

  Preparing metadata (setup.py) ... done
  Created wheel for rulefit: filename=rulefit-0.3.1-py3-none-any.whl size=7799 sha256=83e402bfe589feb6c89f76fe25b1002fef93b969cda9baf2bd0b7f3cbf3da6df
  Stored in directory: /root/.cache/pip/wheels/0d/e7/77/17919d2e417a6589ae77e0722a78f2287c308fca24af14c600
Successfully built rulefit


In [47]:
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier
from rulefit import RuleFit
import time

features = df_filtered.columns
N = X_train.shape[0]

rf = RuleFit(tree_size=4,
             sample_fract='default',
             max_rules=20, # 2000 has good results
             memory_par=0.01,
             tree_generator=None,
             rfmode='classify',
             lin_trim_quantile=0.025,
             lin_standardise=True,
             exp_rand_tree_size=True,
             random_state=42)

# Record the start time
start_time = time.time()

rf.fit(X_train, y_train, feature_names=features)

# Record the end time
end_time = time.time()
# Calculate and print the training time
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

Training time: 338.3270614147186 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [48]:
def far_score(test: list, pred: list) -> float:
  assert len(test) == len(pred)
  false_alarms = 0
  for i in range(len(test)):
    if test[i] == 0 and pred[i] == 1:
      false_alarms += 1

  return false_alarms / test.count(1)

In [49]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

y_pred = rf.predict(X_test)
print(y_test, y_pred)

print('ACCURACY:  ', accuracy_score(y_test, y_pred))
print('PRECISION: ', precision_score(y_test, y_pred))
print('RECALL:    ', recall_score(y_test, y_pred))
print('F1-SCORE:  ', f1_score(y_test, y_pred))
print('FAR:       ', far_score(list(y_test), list(y_pred)))

[1 1 1 ... 0 0 0] [0 1 1 ... 0 0 0]
ACCURACY:   0.9794689296468656
PRECISION:  0.8852459016393442
RECALL:     0.6390532544378699
F1-SCORE:   0.7422680412371134
FAR:        0.08284023668639054


In [50]:
y_pred = rf.predict(X_train)
#y_proba = rf.predict_proba(X)
insample_acc = sum(y_pred == y_train) / len(y_train)

rules = rf.get_rules()

rules = rules[rules.coef != 0].sort_values(by="support")
num_rules_rule = len(rules[rules.type == 'rule'])
num_rules_linear = len(rules[rules.type == 'linear'])
print('NUMBER OF LINEARS: ', num_rules_linear)
print('NUMBER OF RULES: ', num_rules_rule)

NUMBER OF LINEARS:  17
NUMBER OF RULES:  11


In [51]:
rules = rules.sort_values(by='importance', ascending=False)
rules[rules['type']=='rule']

,rule,type,coef,support,importance
25,P_Mean > 100636.60546875 & P_Std <= 275.030075...,rule,0.436690,0.935743,0.107081
26,P_Std > 275.0300750732422 & Vor_850hPa_Mean > ...,rule,-2.064243,0.002677,0.106668
21,Wind_300hPa_Std > 7.747866630554199 & Vor_850h...,rule,-2.051395,0.002677,0.106004
30,Vor_850hPa_Std > 3.694649785757065e-05 & Vor_8...,rule,0.674850,0.024096,0.103487
29,Vor_850hPa_Std > 4.226418423058931e-05 & Vor_8...,rule,0.351073,0.026774,0.056671
19,Vor_850hPa_Mean <= -1.2813274679501774e-05 & W...,rule,0.856863,0.002677,0.044278
31,Vor_850hPa_Mean <= -1.2813274679501774e-05 & W...,rule,0.254181,0.030790,0.043909
22,P_Mean <= 100636.60546875 & Vor_850hPa_Mean > ...,rule,0.483905,0.008032,0.043194
37,Vor_850hPa_Std <= 4.226418423058931e-05 & Vor_...,rule,-0.243866,0.020080,0.034208
24,Vor_850hPa_Std <= 3.694649785757065e-05 & Vor_...,rule,-0.578453,0.002677,0.029891


In [53]:
rules[rules['type']=='linear']

,rule,type,coef,support,importance
4,Wind_Gust_Mean,linear,-0.672495,1.0,1.700896
6,Wind_1000hPa_Mean,linear,0.487862,1.0,1.152919
13,T_1000hPa_Std,linear,-3.925972,1.0,0.867949
2,Vor_850hPa_Mean,linear,-153649.291091,1.0,0.803145
10,Wind_300hPa_Mean,linear,-0.123923,1.0,0.780528
5,Wind_Gust_Std,linear,-0.915947,1.0,0.616388
1,P_Std,linear,0.012629,1.0,0.611667
7,Wind_1000hPa_Std,linear,0.789057,1.0,0.485877
3,Vor_850hPa_Std,linear,56001.257889,1.0,0.451191
15,Air_Density_Std,linear,308.570694,1.0,0.420147


from matplotlib import pyplot as plt
_df_0['coef'].plot(kind='hist', bins=20, title='coef')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['importance'].plot(kind='hist', bins=20, title='importance')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='coef', y='importance', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['support']
  ys = series['coef']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('support', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('support')
_ = plt.ylabel('coef')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['support']
  ys = series['importance']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('support', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('support')
_ = plt.ylabel('importance')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['support']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'support'}, axis=1)
              .sort_values('support', ascending=True))
  xs = counted['support']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_5.sort_values('support', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('support')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_6['coef'].plot(kind='line', figsize=(8, 4), title='coef')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_7['importance'].plot(kind='line', figsize=(8, 4), title='importance')
plt.gca().spines[['top', 'right']].set_visible(False)